# VacationPy

In [51]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [52]:
weather = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather = weather.drop(columns=["Unnamed: 0"])
weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind Speed
0,rikitea,44,PF,1608267008,74,-23.12,-134.97,77.58,17.38
1,butaritari,94,KI,1608267008,81,3.07,172.79,81.25,11.99
2,cape town,0,ZA,1608267008,68,-33.93,18.42,64.99,16.11
3,hansot,19,IN,1608267008,72,21.58,72.80,68.00,9.17
4,alexandria,11,EG,1608266821,65,31.22,29.96,63.91,5.93
...,...,...,...,...,...,...,...,...,...
544,santa isabel do rio negro,85,BR,1608267048,94,-0.41,-65.02,73.38,1.50
545,barstow,1,US,1608267048,42,34.90,-117.02,48.20,14.99
546,tygda,82,RU,1608267048,80,53.12,126.33,-7.15,10.18
547,ode,1,NG,1608267048,87,7.79,5.71,67.68,2.37


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [63]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations and "Humidity" into humidity
locations = weather[["Lat", "Lng"]].astype(float)
humidity = weather['Humidity'].astype(float)

In [62]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=(humidity.max()),
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
#My ideal weather condition is temperature 85 degrees or above, cloudiness less than 15%, and humidity less than 60%
ideal = weather.loc[((weather["Max_Temp"]>= 85) & (weather["Cloudiness"] < 15) & (weather["Humidity"] < 60)),:]
ideal = ideal.dropna(how = 'any')
ideal = ideal.reset_index(drop = True)
ideal

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind Speed
0,narrabri,7,AU,1608267030,43,-30.32,149.78,91.4,12.75
1,geraldton,0,AU,1608267037,29,-28.77,114.60,91.4,21.92


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [56]:
# Store df as hotel_df and add blank column Hotel Name
hotel_df = ideal.copy()
hotel_df["Hotel Name"] = ""

# Set parameters for Google Places search
params = {
    "types": 'lodging',
    "radius": 5000,
    "key": g_key,
}

# Iterate through hotel_df coordinates to return Hotels
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #get lng/lat from df
    lat_coord = row['Lat']
    lng_coord = row['Lng']

    # add location to params dict
    params['location'] = f'{lat_coord},{lng_coord}'
    
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
   
    # extract results
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
            
hotel_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind Speed,Hotel Name
0,narrabri,7,AU,1608267030,43,-30.32,149.78,91.4,12.75,Crossroads Hotel
1,geraldton,0,AU,1608267037,29,-28.77,114.60,91.4,21.92,Broadwater Mariner Resort


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [58]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[hotel for hotel in hotel_info]
)


fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

# Display figure

Figure(layout=FigureLayout(height='420px'))